In [1]:
import cv2
import os
import numpy as np

input_path=r'/data_hs/sjwlab/meixuewen/project2/dataset/localduo_data_redeal/CR2809572LEFT/mlo/DXm.1.2.392.200036.9125.4.0.940547761.101582184.1573523479.png'
output_path=r'equtest.png'

def histogram_equalization(image_path):
    # 读取图像
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    print(img.shape)
    if img is None:
        print(f"Error: Unable to load image at {image_path}.")
        return None

    # 直方图均衡化
    equalized_img = cv2.equalizeHist(img)

    return equalized_img

cv2.imwrite(output_path,histogram_equalization(input_path))


(2963, 2266)


True

In [ ]:
def dilate_image(img, kernel_size=(7, 7), iterations=7):
    """
    对图像进行膨胀操作。

    :param image_path: 图像的路径
    :param kernel_size: 膨胀操作的内核大小，默认为(5, 5)
    :param iterations: 膨胀操作的迭代次数，默认为1
    :return: 膨胀后的图像
    """
    # # # 读取图像
    # img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("Image not found or unable to be read.")
    
    # 创建结构元素，用于膨胀操作
    kernel = np.ones(kernel_size, np.uint8)
    
    # 执行膨胀操作
    dilated_img = cv2.dilate(img, kernel, iterations=iterations)
    
    return dilated_img

def find_and_draw_largest_contour_with_min_rect(img):
    # 读取图像
    # img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found or unable to be read.")
    
    # # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # gray=img
    # 应用阈值操作以获取二值图像
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # 寻找轮廓
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # 检查是否有轮廓被找到
    if not contours:
        raise ValueError("No contours found.")
    
    # 找到最大的轮廓
    largest_contour = max(contours, key=cv2.contourArea)
    
    # 计算最小外接矩形
    rect = cv2.minAreaRect(largest_contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    
    # 在原图上绘制最小外接矩形（非实心）
    cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
    
    return img
def crop_image_edges(img, crop_size=3):
    """
    对三通道图像进行四周裁剪。

    :param image_path: 图像的路径
    :param crop_size: 裁剪的像素大小，默认为30
    :return: 裁剪后的图像
    """
    # 读取图像
    # img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found or unable to be read.")
    
    # 检查图像是否为三通道
    if len(img.shape) != 3 or img.shape[2] != 3:
        raise ValueError("Input image is not a three-channel image.")
    
    # 裁剪图像四周
    cropped_img = img[crop_size:-crop_size, crop_size:-crop_size]
    
    return cropped_img
input_path=r'/data_hs/sjwlab/meixuewen/project3/zstack/redeal_localduo_data/CR2209833RIGHT/cc_equ/DXm.1.2.392.200036.9125.4.0.1880045876.2499806568.264908388.png'

img=cv2.imread(input_path)
print(img.shape)
cv2.imwrite('equtest.png',find_and_draw_largest_contour_with_min_rect(dilate_image(crop_image_edges(img))))




In [4]:
import cv2
import numpy as np

def dilate_image(img, kernel_size=(7, 7), iterations=7):
    kernel = np.ones(kernel_size, np.uint8)
    dilated_img = cv2.dilate(img, kernel, iterations=iterations)
    return dilated_img

def find_and_draw_largest_contour_with_min_rect(img,imgorg):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        raise ValueError("No contours found.")
    
    largest_contour = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(largest_contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    
    # 在原图上绘制最小外接矩形（非实心）
    # cv2.drawContours(imgorg, [box], 0, (0, 255, 0), 2)
    
    # 根据最小外接矩形的坐标裁剪图像
    x_min, y_min = np.min(box, axis=0)
    x_max, y_max = np.max(box, axis=0)
    cropped_img = imgorg[y_min:y_max, x_min:x_max]
    
    return cropped_img

def crop_image_edges(img, crop_size=3):
    if len(img.shape) != 3 or img.shape[2] != 3:
        raise ValueError("Input image is not a three-channel image.")
    
    cropped_img = img[crop_size:-crop_size, crop_size:-crop_size]
    return cropped_img

input_path = r'/data_hs/sjwlab/meixuewen/project2/dataset/localduo_data/images_two_many/CR1166776LEFT/cc/DXm.1.2.392.200036.9125.4.0.336576991.3568638720.958488888.png'

img = cv2.imread(input_path)
if img is None:
    raise ValueError("Image not found or unable to be read.")

# 先裁剪边缘，再膨胀，最后找到最大轮廓并裁剪
img_cropped = crop_image_edges(img)
img_dilated = dilate_image(img_cropped)
img_with_min_rect = find_and_draw_largest_contour_with_min_rect(img_dilated,img_cropped)

print(img_with_min_rect.shape)
cv2.imwrite('equtest.png', img_with_min_rect)


(1205, 726, 3)


/tmp/ipykernel_3612031/10656147.py:20: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


True

批量化

In [69]:
import cv2
import numpy as np

def dilate_image(img, kernel_size=(7, 7), iterations=7):
    kernel = np.ones(kernel_size, np.uint8)
    dilated_img = cv2.dilate(img, kernel, iterations=iterations)
    return dilated_img

def find_and_draw_largest_contour_with_min_rect(img,imgorg):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        raise ValueError("No contours found.")
    
    largest_contour = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(largest_contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    
    # 在原图上绘制最小外接矩形（非实心）
    # cv2.drawContours(imgorg, [box], 0, (0, 255, 0), 2)
    
    # 根据最小外接矩形的坐标裁剪图像
    x_min, y_min = np.min(box, axis=0)
    x_max, y_max = np.max(box, axis=0)
    cropped_img = imgorg[y_min:y_max, x_min:x_max]
    
    return cropped_img

def crop_image_edges(img, crop_size=3):
    if len(img.shape) != 3 or img.shape[2] != 3:
        raise ValueError("Input image is not a three-channel image.")
    
    cropped_img = img[crop_size:-crop_size, crop_size:-crop_size]
    return cropped_img

input_path = r'/data_hs/sjwlab/meixuewen/project3/zstack/redeal_localduo_data/CR2209833RIGHT/cc_equ'
out_path=r'/data_hs/sjwlab/meixuewen/project3/zstack/redeal_localduo_data/CR2209833RIGHT/cc_equ_de'
os.makedirs(out_path,exist_ok=True)
for file in os.listdir(input_path):
    pathim=os.path.join(input_path,file)
    pathout=os.path.join(out_path,file)

    img = cv2.imread(pathim)
    if img is None:
        raise ValueError("Image not found or unable to be read.")

    # 先裁剪边缘，再膨胀，最后找到最大轮廓并裁剪
    img_cropped = crop_image_edges(img)
    img_dilated = dilate_image(img_cropped)
    img_with_min_rect = find_and_draw_largest_contour_with_min_rect(img_dilated,img_cropped)

    print(img_with_min_rect.shape)
    cv2.imwrite(pathout, img_with_min_rect)


/tmp/ipykernel_300910/563496533.py:20: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


(1077, 341, 3)
(1071, 343, 3)
(1037, 343, 3)
(1037, 342, 3)
(1077, 341, 3)
(1049, 343, 3)
(1111, 340, 3)
(1107, 342, 3)
(1074, 342, 3)
(1074, 342, 3)
(1036, 343, 3)
(1076, 341, 3)
(1076, 341, 3)
(1077, 342, 3)
(1037, 343, 3)
(1036, 343, 3)
(1050, 343, 3)
(1031, 342, 3)
(2748, 936, 3)
(1044, 343, 3)
(1108, 341, 3)
(1035, 343, 3)
(1047, 343, 3)
(1077, 345, 3)
(1110, 342, 3)
(1075, 342, 3)
(1036, 341, 3)
(1044, 343, 3)
(1111, 342, 3)
(1069, 343, 3)
(1067, 343, 3)
(1043, 343, 3)
